In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [35]:
import scipy.linalg as lin
import numpy.linalg as linNum
import numpy as numpy
#from sympy import *
from sympy import Matrix
#from sage.interfaces.phc import phc
import phcpy as phc
import ipywidgets as widgets

from sage.plot.plot3d.plot3d import axes
from sage.plot.colors import rainbow
numpy.seterr('raise')

numpy.set_printoptions(precision=3,suppress=True,formatter={'float': '{: 0.3f}'.format}) #comment this for debug
# maybe use numpy.negative to negative numbers

class MyError(Exception):
    pass

#var('x11 x12 x13 x14')
wg=''
#ag=1
#firstThetag=pi
#secondThetag=pi/2
#bg=2
#firstPhig=pi
#secondPhig=pi
arrayOfPoints = {} #used for animation, the order of words is by first in line

def drawLinesHelper(lines,ns,**options):
    resu = []
    preceeding = Graphics()
    for j in range(1,len(lines)):
        x1, y1,z1 = lines[j-1]
        x2, y2,z2 = lines[j]
        
        dx = float(x2 - x1)/float(ns)
        dy = float(y2 - y1)/float(ns)
        dz = float(z2 - z1)/float(ns)
        for i in range(ns):
            xx = x1 + (i+1)*dx
            yy = y1 + (i+1)*dy
            zz = z1 + (i+1)*dz
            resu.append(preceeding + line([(x1,y1,z1), (xx,yy,zz)], **options))
        preceeding = resu[-1]
    return resu
@parallel
def drawlines(ns,endIndex, **options):
    r"""
    Input:
    - ``lines``: a list of lines, each line being defined
      by its endpoints as ``[(x1,y1,z1), (x2,y2,z2)]``
    - ``ns``: the number of division of each line for the 
      animation
    - ``**options``: any option to pass to the graphic 
      command ``line``, like ``color`` or ``thickness``
    """
    resu = []
    preceeding = Graphics()
    anim = Graphics()
    colorsDict = dict(zip(arrayOfPoints.keys(),rainbow(len(arrayOfPoints.keys()))))
    for key, word in arrayOfPoints.items():
        #lines = numpy.array_split(word,2)
        #word = [list(ele) for ele in word] #make list of tuples to list of lists
        #lines = word[:endIndex]#was lines[:endIndex]
      
        resu = drawLinesHelper(word,ns, color=colorsDict[key], thickness=2)
        anim += animate(resu).interactive()
        #print(len(lines)) #debug?
    return anim
def customAtan(y,x):
    v=numpy.mod(numpy.arctan2(y,x)+ 2*numpy.pi,2*numpy.pi)
    return v
def svdcomposition(b,firstPhi,secondPhi):
    #rf_low = RealField(12)
    #print("product") DEBUG
    #print(ux@sx@vhx)
    #print("ux before numpy svd")
    #print(ux)
    #print("vhx before numpy svd")
    #print(vhx)
    uy = numpy.array([[numpy.cos(firstPhi),-numpy.sin(firstPhi)],[numpy.sin(firstPhi),numpy.cos(firstPhi)]]) #matrix(rf_low,2,2,[cos(firstPhi),-sin(firstPhi),sin(firstPhi),cos(firstPhi)])
    sy = numpy.array([[b,0],[0,1/b]])#matrix(rf_low,2,2,[b,0,0,1/b])
    vhy = numpy.array([[numpy.cos(secondPhi),-numpy.sin(secondPhi)],[numpy.sin(secondPhi),numpy.cos(secondPhi)]]) #matrix(rf_low,2,2,[cos(secondPhi),-sin(secondPhi),sin(secondPhi),cos(secondPhi)])
    return uy@sy@vhy

    
def makeunitarydeterminant1(u,v,msign,s):
    """
    v means the V transpose matrix
    msign is the sign of the matrix
    diagonal matrix s
     ---------
     | a | b |
     | c | d | 
     ---------
     
     make ad-bc = 0
    """
    
    negativePivotLead = numpy.array([[-1,0],[0,1]])#matrix(RealField(12),2,2,[-1,0,0,1])
    negativePivotFollow = numpy.array([[1,0],[0,-1]])#matrix(RealField(12),2,2,[1,0,0,-1])
    
    top = negativePivotLead @ v
    bottom = negativePivotFollow @ v
    left =   u @negativePivotLead
    right = u @ negativePivotFollow 
    leftRight =  u @negativePivotLead @ negativePivotFollow
    topBottom =   negativePivotLead @ negativePivotFollow @ v
    
    array = numpy.array([[linNum.det(left),linNum.det(right),linNum.det(right),linNum.det(left),linNum.det(left),linNum.det(right),linNum.det(leftRight),linNum.det(leftRight),linNum.det(leftRight)],[linNum.det(top),linNum.det(bottom),linNum.det(topBottom),linNum.det(top),linNum.det(bottom),linNum.det(topBottom),linNum.det(topBottom),linNum.det(top),linNum.det(bottom)]])
    array = numpy.sum(array,axis=0)
    arrayMatrixRotation = numpy.array([[left,right,leftRight,right,left,left,right,leftRight,leftRight],[s,s,s,s,s,s,s,s,s],[top,bottom,topBottom,top,bottom,topBottom,topBottom,top,bottom]])
    #arrayProduct = numpy.prod(arrayMatrixRotation,axis=0)#matrix multiplication gives differing result than @
    #if (numpy.sign(u[0,0]) == 1 and numpy.sign(u[0,1]) == 1 and numpy.sign(u[1,0]) == 1 and numpy.sign(u[1,1])== -1) or  (numpy.sign(u[0,0]) == -1 and numpy.sign(u[0,1]) == 1 and numpy.sign(u[1,1]) == 1 and numpy.sign(u[1,0]) == 1) :
    #    array[0] = 0
    if(numpy.sign(u[1,0]) == 1 and (math.isclose(numpy.abs(u[0,0]),1,abs_tol=1e-16) == False)): #isclose is used when value is close to 1
        array[0] = 0
    if(numpy.sign(u[0,0]) == 1 and (math.isclose(numpy.abs(u[0,0]),1,abs_tol=1e-16))): 
        array[0] = 0
        
    arrayProduct = []
    boolArray = []
    for i in range(arrayMatrixRotation.shape[1]):
        arrayProduct.append(arrayMatrixRotation[0,i] @ arrayMatrixRotation[1,i] @ arrayMatrixRotation[2,i])
        
    for elem in arrayProduct:
        #print("elem\n")
        #print(numpy.sign(elem))
        #print("msign\n")
        #print(msign)
        if numpy.array_equal(numpy.sign(elem),msign):
            boolArray.append(True);
        else:
            boolArray.append(False);
    
    max_index = array.argmax()
 
   # print(msign) #debug
    #print(numpy.sign(arrayProduct))
    #print(array)
    #print(boolArray)
    if(max_index == 0 and boolArray[0] == True):
        v = top
        u = left
    elif(max_index == 1 and boolArray[1] == True):
        v = bottom
        u = right
    elif(max_index == 2 and boolArray[2] == True):
        v = top
        u = right
    elif(max_index == 3 and boolArray[3] == True):
        v = bottom
        u = left
    elif(max_index == 4 and boolArray[4] == True):
        v = topBottom
        u = left
    elif(max_index == 5 and boolArray[5] == True):
        v = topBotom
        u = right
    elif(max_index == 6 and boolArray[6] == True):
        v = top
        u = leftRight
    elif(max_index == 7 and boolArray[7] == True):
        v = bottom
        u = leftRight
    elif(max_index == 8 and boolArray[8] == True):
        v = topBottom
        u = leftRight
    
       
       
  
        
    return u,v;
def findRoots(wordMatrix,zM):
    r"""
    Solve for X matrix
    """
    
    system = [str(wordMatrix[0,0]-zM[0,0])+';', str(wordMatrix[0,1]-zM[0,1])+';',str(wordMatrix[1,0]-zM[1,0])+';',str(wordMatrix[1,1]-zM[1,1])+';']
    #print(system)#debug
    v=phc.solver.solve(system)
   
    if v is None:
        return None
    realsols = phc.solutions.is_real(v[0],0) #boolean if solutions are real within a tolerance
    if not realsols:
        return None
    v = phc.solutions.strsol2dict(v[0])
    
    return numpy.array([[ v['x11'].real, v['x12'].real],[ v['x21'].real, v['x22'].real]]) 
def retrieveWords(w,xM,yM,zM,printWord=false):
    
    wordArray = w.split(",")
  
    wordArray = list(filter(lambda x: (''.join(x.split())),wordArray))
   
    wordArray = list(filter(lambda x: (set(x) - set('xyXY') == set()), wordArray))
    wordArray = list(set(wordArray))
    
    wordTuples = []
   
    hasX = False;#phcpack won't accept if there are no variables
    for word in wordArray:
    
        wordMatrix = numpy.array([[1,0],[0,1]])#matrix(RealField(12),2,2,[1,0,0,1])##
        for character in word:
            if character == 'x':  
                wordMatrix = wordMatrix @ xM
                hasX=True
            elif character == 'y': 
                wordMatrix = wordMatrix @ yM
            elif character == 'X':
                wordMatrix = wordMatrix @ numpy.array(Matrix(xM).inv())#xM.inverse()#linNum.inv(xM)
            elif character == 'Y':  
                wordMatrix = wordMatrix @ numpy.array(Matrix(yM).inv())#yM.inverse()#linNum.inv(yM)
            else:
                raise MyError("Unrecognized Character");
        #print(wordMatrix) debug
        if not hasX:
            continue;
        if printWord:
            #display(widgets.HTMLMath("%s = z_{0} := %s = %s$"%(r"$"+word, latex(matrix(wordMatrix.tolist())),latex(matrix(zM.tolist())))))
         
            display(widgets.HTMLMath("<br />%s = z_{0} := %s = %s$"%(r"$"+word, latex(matrix(wordMatrix.tolist())),latex(matrix(zM.tolist())))))
            
        wordMatrix = findRoots(wordMatrix,zM)
        if wordMatrix is None:#check if complex
            continue;
        u,s,vh = lin.svd(wordMatrix,lapack_driver='gesvd')#lin.svd(wordMatrix.numpy())#use scip.svd or numpy.svd
        s = numpy.diag(s)#make s diagonal
        #print("before u") #DEBUG
        #print(u)
        #print("before vh")
        #print(vh)
        u,vh = makeunitarydeterminant1(u,vh,numpy.sign(wordMatrix),s)
        #print("after u")
        #print(u)#test
        #print("after vh")
        #print(vh)#test
        #vh = makeunitarydeterminant1(vh)
       
       
        
      
            
        wordTuples.append((word,s[0,0],(customAtan(u[1][0],u[0][0])),customAtan(vh[1][0],vh[0][0]),u[1][0],u[0][0],vh[1][0],vh[0][0]))
        
    return wordTuples
def build_frame(b,firstTheta,secondTheta,c,firstPhi,secondPhi):
    global arrayOfPoints
    yM=svdcomposition(b,firstTheta,secondTheta)
    zM=svdcomposition(c,firstPhi,secondPhi)
    M = Manifold(3, 'R^3', r'\RR^3', structure='topological', start_index=0)
    spher.<r,th,ph1> = M.chart(r'r:(0,+oo) th:(0,2*pi):periodic:\theta ph1:(0,2*pi):periodic:\phi_1')
    cart.<x,y,z>=M.chart()
    spher.transition_map(cart,[th,ph1,r])
   
    y = M.point((b,firstTheta,secondTheta), chart=spher,name = 'y')
    arrayOfPoints.setdefault('yp',[]).append((firstTheta,secondTheta,b))
    z = M.point((c,firstPhi,secondPhi), chart=spher,name = 'z')
    arrayOfPoints.setdefault('zp',[]).append((firstPhi,secondPhi,c))
    R.<x11,x12,x21,x22> = PolynomialRing(RDF,4)
    xM = numpy.array([[x11,x12],[x21,x22]])
    wordsArray = retrieveWords(wg,xM,yM,zM,false)
    G = y.plot(cart,color = 'green')+z.plot(cart, color = 'yellow')+axes(1,0.5,color='black')
    
       
    #plot words
    if wordsArray is not None:
         for word,d,firstPsi,secondPsi,fy,fx,sy,sx in wordsArray:
            G += M.point((d,firstPsi,secondPsi),chart = spher, name = word).plot(cart,color = 'orange')
            arrayOfPoints.setdefault(word,[]).append((firstPsi,secondPsi,d))
            #print("First Theta Error:(firsttheta-firstPsi) " + str(firstTheta - firstPsi) +" 1the " + str(firstTheta) + " 1psi " + str(firstPsi) + " y " + str(fy) + " x " +str(fx))
            #print("Second Theta Error " + str(secondTheta - secondPsi) +" " + str(secondTheta) + " " + str(secondPsi) + " " + str(fy) + " " +str(fx))
    
    return G
                                

    
    

    
#first angle is [0,pi) and second angle is [0,2*pi]. Scalar(a,b) is greater than 0
@interact(layout=dict(top=[['w']],left=[['b'],['firstTheta'],['secondTheta']],right=[['c','firstPhi','secondPhi']]))



def main(w='xy',b=slider(1.1,100,0.1,2,label='$b_{0}$'),firstTheta=slider(0,3.14,0.01,3.14,label=r'\(\theta_1\)'),secondTheta=slider(0,2*3.14,0.01,3.14,label=r'\(\theta_2\)'),c=slider(1.1,100,0.1,2,label='$c_{0}$'),firstPhi=slider(0,3.14,0.01,3.14,label=r'\(\phi_1\)'),secondPhi=slider(0,2*3.14,0.01,3.14,label=r'\(\phi_2\)'),auto_update=False):


    global wg,ag,firstThetaq,secondThetag,bg,firstPhg,secondPhig,arrayOfPoints#global is used to help build_frame

    wg=w
   
 
   
    yM=svdcomposition(b,firstTheta,secondTheta)
    zM=svdcomposition(c,firstPhi,secondPhi)
    #pretty_print(html("$y = %s$  $z= %s$"%(latex(matrix(zM.tolist())), latex(matrix(yM.tolist())))))
   # pretty_print(html("Determinant  is   %s"%(latex(d)))) #uncomment to print determinant

    M = Manifold(3, 'R^3', r'\RR^3', structure='topological', start_index=0)
    spher.<r,th,ph1> = M.chart(r'r:(0,+oo) th:(0,2*pi):periodic:\theta ph1:(0,2*pi):periodic:\phi_1')
    cart.<x,y,z>=M.chart()
    spher.transition_map(cart,[th,ph1,r])

    #print('x from input') #debug
    #print(a)
    #print(firstTheta)
    #print(secondTheta)
    #print('y from input')
    #print(b)
    #print(firstPhi)
    #print(secondPhi)

    yPoint = M.point((b,firstTheta,secondTheta), chart=spher,name = 'y0')
    zPoint = M.point((c,firstPhi,secondPhi), chart=spher,name = 'z0')
    R.<x11,x12,x21,x22> = PolynomialRing(RDF,4)
    xM = numpy.array([[x11,x12],[x21,x22]])
    wordsArray = retrieveWords(w,xM,yM,zM,true)
    G = yPoint.plot(cart,color = 'green')+zPoint.plot(cart, color = 'yellow')+axes(1,0.5,color='black')
       
    #plot words
    if wordsArray is not None:
         for word,d,firstPsi,secondPsi,fy,fx,sy,sx in wordsArray:
            G += M.point((d,firstPsi,secondPsi),chart = spher, name = word).plot(cart,color = 'orange')
            #print(word + " from svd") #debug
            #print(c)
            #print(firstPsi)
            #print(secondPsi)
   

    G.show(axes_label=['phi_1','phi_2','c'])#latex not supported yet for axex_label https://trac.sagemath.org/ticket/30226 todo
    display(widgets.HTMLMath('<h3>Animation where %s is not constant'%(r'\(\theta_1\)')))
    #Let's animate!
    frames = [build_frame(b,i,secondTheta,c,firstPhi,secondPhi) for i in (0, 0.2, 0.3, .., 3.14)]# could do for a,b in zip(range1,range2)
    plot = animate(frames).interactive() + drawlines(1,0)
    plot.show(delay=10, auto_play=False, projection='orthographic',format='mp4')
    arrayOfPoints.clear()

    #pretty_print(html('<h3>Animation where %s is not constant'%(r'\(\theta_2\)')))
    #frames = [build_frame(b,firstTheta,i,c,firstPhi,secondPhi) for i in (0, 0.2, 0.3, .., 2*numpy.pi)]# could do for a,b in zip(range1,range2)
  
    #plot = animate(frames).interactive() + drawlines(1,0)
    #plot.show( delay=10, auto_play=False, projection='orthographic',format='mp4')
    #arrayOfPoints.clear()#clear points because interactive will not
    #show(drawlines(1,0), delay=10, auto_play=False, projection='orthographic')
    #clear arrayOfPoints after an animation with any of the variables
    #pretty_print(html('<h3>Animation where firstTheta and firstPhi are not constant'))
    #changes below to pi from 2*pi
    #frames = [build_frame(b,j,k,c,firstPhi,secondPhi) for j,k in zip((0.1, 0.2, 0.3, .., numpy.pi),(0.1, 0.2, 0.3, .., numpy.pi))]# could do for a,b in zip(range1,range2)
    #plot = animate(frames).interactive()
    #show(plot, delay=10, auto_play=False, projection='orthographic')

Manual interactive function <function main at 0x7f939699be50> with 7 widgets
  w: Text(value='xy', description…